<H2>Create an I girder model in shell elements</H2>

Create a girder class to hold the section dimensions for each segment
We'll create points in the LUSAS model for each section and then use the points to create surfaces joining them.

In [86]:
class Girder:
    def __init__(self, tfb:float, tfthk:float, dw:float, tw:float, bfb:float, bfthk:float):
        self.top_flange_breadth = tfb
        self.top_flange_thk = tfthk
        self.web_depth = dw
        self.web_thk = tw
        self.bottom_flange_breadth = bfb
        self.bottom_flange_thk = bfthk
        # We'll save the points created in the model in these lists so we can later use them to define surfaces
        self.bf_points = []
        self.tf_points = []


Define a list of segment lengths so the girder can have changes in sections

In [87]:
segments_lengths = [6000, 5000, 6000]

Create a list of section definitions, we'll need one per segment + 1 for the end.

In [88]:
sections = []
sections.append(Girder(500, 30, 1500, 15, 600, 40))
sections.append(Girder(500, 50, 1500, 10, 600, 60))
sections.append(Girder(500, 30, 1500, 15, 600, 40))
sections.append(Girder(500, 30, 1500, 15, 600, 40))
# Check we have defined one more section then segment lengths
assert(len(segments_lengths) == len(sections)-1)

Connect to LUSAS Modeller

In [89]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.0")
db = lusas.database() # Get a reference to the current model database for convenience - note a blank model needs to be open


Create a helper function to create a point in the model

In [90]:
def create_point_in_modeller(x:float, y:float, z:float) -> 'IFPoint':
    return win32.CastTo(db.createPoint(x, y, z), "IFPoint")

Create all the points for each section

In [91]:
s:Girder
x = 0
for i in range(0, len(sections)):
    # Determine the longitudinal position (x coord) for each section
    if i > 0 : x += segments_lengths[i-1]
    # Determine the section to use at each longitudinal position
    s = sections[i]
    # Create the bottom flange points
    s.bf_points.append(create_point_in_modeller(x, +s.bottom_flange_breadth/2, 0.0))
    s.bf_points.append(create_point_in_modeller(x, 0.0,                        0.0))
    s.bf_points.append(create_point_in_modeller(x, -s.bottom_flange_breadth/2, 0.0))
    # Create the top flange points
    s.tf_points.append(create_point_in_modeller(x, +s.top_flange_breadth/2, s.web_depth))
    s.tf_points.append(create_point_in_modeller(x, 0.0,                     s.web_depth))
    s.tf_points.append(create_point_in_modeller(x, -s.top_flange_breadth/2, s.web_depth))

Now join up the segment points. First create helper function to create surfaces and assign a geometric thickness attribute

In [92]:
def create_surface_in_lusas(pnts: list, thk:float, ecc:float, group:str):

    geometryData = lusas.geometryData()
    geometryData.setAllDefaults()
    geometryData.setCreateMethod("coons")
    geometryData.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet()
    for p in pnts:
        obs.add(p)

    objs = obs.createSurface(geometryData).getObjects("Surface")
    grp = db.getGroupByName(group)
    grp.add(objs, "Surfaces")

    name = f"{group} ({thk:.1f}mm)"

    db.createGeometricSurface(name).setSurface(thk, ecc).assignTo(objs)


Create groups to contain the girder surfaces

In [93]:
db.createEmptyGroup("Top Flange")
db.createEmptyGroup("Web")
db.createEmptyGroup("Bottom Flange")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.0.IFGroup instance at 0x2253936993008>

Here we'll create the surfaces between each section definition using the points we created earlier

In [94]:
for i in range(0, len(segments_lengths)):

    s1:Girder = sections[i]   # Section definition at the start of the segment
    s2:Girder = sections[i+1] # Section definition at the end of the segment

    create_surface_in_lusas([ s1.bf_points[0], s2.bf_points[0], s2.bf_points[1], s1.bf_points[1] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
    create_surface_in_lusas([ s1.bf_points[1], s2.bf_points[1], s2.bf_points[2], s1.bf_points[2] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
    create_surface_in_lusas([ s1.bf_points[1], s1.tf_points[1], s2.tf_points[1], s2.bf_points[1] ], s1.web_thk,           0.0, "Web")
    create_surface_in_lusas([ s1.tf_points[0], s2.tf_points[0], s2.tf_points[1], s1.tf_points[1] ], s1.top_flange_thk,    0.0, "Top Flange")
    create_surface_in_lusas([ s1.tf_points[1], s2.tf_points[1], s2.tf_points[2], s1.tf_points[2] ], s1.top_flange_thk,    0.0, "Top Flange")

We'll need to create and assign a shell mesh attribute

In [95]:
# Create the mesh attribute
mesh_attr = db.createMeshSurface("SMsh1")
mesh_attr.setRegular("QTS4", 0, 0, True)
mesh_attr.setValue("allowIrregular", True)

# Assign to all 
assignment = lusas.assignment().setAllDefaults().setLoadset(1)
mesh_attr.assignToAll(assignment)

assignment = None # Prevent notebook holding on to objects
mesh_attr  = None

And a material attribute

In [96]:
# Create and assign the steel material attribute, we'll do this all inline so we dont hold onto any references
db.createIsotropicMaterial("Steel", 200_000, 0.3, 7.8, 10e-6).assignToAll(lusas.assignment().setAllDefaults().setLoadset(1))

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.0.IFAttribute instance at 0x2253976691872>

Support attributes

In [97]:
pinned = db.createSupportStructural("Fixed in XYZ").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in YZ").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")




Release all of the references to the point objects now we no longer need them. Note for a normal script that runs start to end these will be released automatically. For a program that ho0lds state, such as is the case in this notebook, it's good practice to release any references

In [98]:
for s in sections:
    s.bf_points = None
    s.tf_points = None

Set the model to display the assigned plate thicknesses in an isometric orientation

In [99]:
lusas.view().geometry().autoColourByAttributes("Geometric", True)
lusas.view().setIsometric()